<a href="https://colab.research.google.com/github/cryssoga/DSWP/blob/master/Li%C3%A7%C3%A3o/Kaggle_Churn_Challenge_Logistic_Regression_16nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Labdata Churn Challenge 2020

Try to predict whether a client will churn the telco service
Cada linha representa um cliente e cada coluna representa uma informação a respeito daquele cliente.

Os dados incluem os seguintes grupos de variáveis:

•	Churn: representa se um cliente cancelou ou não os serviços contratados

•	Serviços que cada cliente contratou - phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies

•	Informações a respeito da conta do cliente – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges

•	Informações demográficas – gender, age range, and if they have partners and dependents

The raw data contains 7043 rows (customers) and 21 columns (features).

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# url dos arquivos baixados do site https://www.kaggle.com/c/labdata-churn-challenge-2020/data e uploaded em github

url_train = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/train.csv'
url_test = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/test.csv'
url_sample_submission = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/sample_submission.csv'

In [6]:
# carregando em dataframes:

df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)
df_sample_submission = pd.read_csv(url_sample_submission)

In [298]:
df_train.shape

(5634, 21)

In [299]:
df_test.shape

(1409, 20)

In [300]:
1409/(1409+5634)      # test: 20%, train: 80%

0.20005679397983814

In [301]:
df_sample_submission.shape

(1409, 2)

In [ ]:
df_train.head(1)  

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0


In [ ]:
df_test.head(1)

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.0,445.3


In [ ]:
df_sample_submission.head(1)

,id,Churn
0,5027,0


In [ ]:
df_train.isna().sum()

id                    0
gender                0
SeniorCitizen         0
Partner               0
Dependents          218
tenure              461
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod        99
MonthlyCharges        0
TotalCharges          0
Churn                 0
dtype: int64

In [ ]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5634 entries, 0 to 5633
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5634 non-null   int64  
 1   gender            5634 non-null   object 
 2   SeniorCitizen     5634 non-null   int64  
 3   Partner           5634 non-null   object 
 4   Dependents        5416 non-null   object 
 5   tenure            5173 non-null   float64
 6   PhoneService      5634 non-null   object 
 7   MultipleLines     5634 non-null   object 
 8   InternetService   5634 non-null   object 
 9   OnlineSecurity    5634 non-null   object 
 10  OnlineBackup      5634 non-null   object 
 11  DeviceProtection  5634 non-null   object 
 12  TechSupport       5634 non-null   object 
 13  StreamingTV       5634 non-null   object 
 14  StreamingMovies   5634 non-null   object 
 15  Contract          5634 non-null   object 
 16  PaperlessBilling  5634 non-null   object 


In [ ]:
df_train[df_train['TotalCharges']==' ']

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
161,6755,Male,0,No,Yes,0.0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,,0
416,5219,Male,0,Yes,Yes,0.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,0
1766,6671,Female,0,Yes,Yes,0.0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,0
2145,937,Female,0,Yes,Yes,0.0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,0
3375,3827,Male,0,Yes,Yes,0.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,0
3404,1083,Male,0,Yes,Yes,0.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,0
4008,3332,Male,0,Yes,Yes,0.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,0
5151,1341,Female,0,Yes,Yes,0.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,0


In [ ]:
df_test[df_test['TotalCharges']==' ']

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
313,4381,Female,0,Yes,Yes,0.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1063,489,Female,0,Yes,Yes,0.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
1297,754,Male,0,No,Yes,0.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,


In [ ]:
df_train['PaymentMethod'].value_counts()

Electronic check             1887
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
Name: PaymentMethod, dtype: int64

In [ ]:
df_test.isna().sum()

id                    0
gender                0
SeniorCitizen         0
Partner               0
Dependents           41
tenure              118
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod        30
MonthlyCharges        0
TotalCharges          0
dtype: int64

In [ ]:
df_test['Dependents'].value_counts()

No     953
Yes    415
Name: Dependents, dtype: int64

In [ ]:
df_test['PaymentMethod'].value_counts()

Electronic check             430
Credit card (automatic)      329
Mailed check                 313
Bank transfer (automatic)    307
Name: PaymentMethod, dtype: int64

In [7]:
def trata_col(df):
  df['Dependents'].fillna('No', inplace=True)
  df['PaymentMethod'].fillna('Electronic check',inplace = True)
  df['tenure'].fillna(df['tenure'].median(),inplace = True)
  df['TotalCharges'] = df['TotalCharges'].replace(' ', '0')
  df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
  df['TotalCharges'] = df['TotalCharges'].replace(0, df['TotalCharges'].median())
  df['SeniorCitizen'] = df['SeniorCitizen'].replace(0,'0')
  df['SeniorCitizen'] = df['SeniorCitizen'].replace(1,'1')

In [8]:
def cria_total_serv(df):
  total_serv = []
  col_reflexo = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
  linha = 0
  while linha < df.shape[0]:
    qtde=0
    if df['InternetService'][linha]=='No':
      qtde = 0
    else:
      for i in col_reflexo:
        if df[i][linha] == 'Yes':
          qtde = qtde + 1
    total_serv.append(qtde)
    linha+=1
  df['total_serv']=total_serv

In [9]:
trata_col(df_train)
trata_col(df_test)
cria_total_serv(df_train)
cria_total_serv(df_test)

In [305]:
df_train.isna().sum()

id                  0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
total_serv          0
dtype: int64

In [306]:
df_test.isna().sum()

id                  0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
total_serv          0
dtype: int64

In [307]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5634 entries, 0 to 5633
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5634 non-null   int64  
 1   gender            5634 non-null   object 
 2   SeniorCitizen     5634 non-null   object 
 3   Partner           5634 non-null   object 
 4   Dependents        5634 non-null   object 
 5   tenure            5634 non-null   float64
 6   PhoneService      5634 non-null   object 
 7   MultipleLines     5634 non-null   object 
 8   InternetService   5634 non-null   object 
 9   OnlineSecurity    5634 non-null   object 
 10  OnlineBackup      5634 non-null   object 
 11  DeviceProtection  5634 non-null   object 
 12  TechSupport       5634 non-null   object 
 13  StreamingTV       5634 non-null   object 
 14  StreamingMovies   5634 non-null   object 
 15  Contract          5634 non-null   object 
 16  PaperlessBilling  5634 non-null   object 


In [10]:
y_train = df_train['Churn']
y_test = df_sample_submission['Churn']

In [11]:
def cria_dummies(df):
  ohe=pd.get_dummies(df,drop_first=True)
  ohe.columns = ohe.columns.str.replace(' ', '_')
  ohe.columns = ohe.columns.str.replace('(','')
  ohe.columns = ohe.columns.str.replace(')','')
  return ohe

In [14]:
df_train=cria_dummies(df_train)

In [15]:
df_train.columns

Index(['id', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn', 'total_serv',
       'gender_Male', 'SeniorCitizen_1', 'Partner_Yes', 'Dependents_Yes',
       'PhoneService_Yes', 'MultipleLines_No_phone_service',
       'MultipleLines_Yes', 'InternetService_Fiber_optic',
       'InternetService_No', 'OnlineSecurity_No_internet_service',
       'OnlineSecurity_Yes', 'OnlineBackup_No_internet_service',
       'OnlineBackup_Yes', 'DeviceProtection_No_internet_service',
       'DeviceProtection_Yes', 'TechSupport_No_internet_service',
       'TechSupport_Yes', 'StreamingTV_No_internet_service', 'StreamingTV_Yes',
       'StreamingMovies_No_internet_service', 'StreamingMovies_Yes',
       'Contract_One_year', 'Contract_Two_year', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit_card_automatic', 'PaymentMethod_Electronic_check',
       'PaymentMethod_Mailed_check'],
      dtype='object')

In [16]:
df_test=cria_dummies(df_test)

In [544]:
df_train=pd.get_dummies(df_train,drop_first=True)
df_train.columns = df_train.columns.str.replace(' ', '_')
df_train.columns = df_train.columns.str.replace('(','')
df_train.columns = df_train.columns.str.replace(')','')
df_test=pd.get_dummies(df_test,drop_first=True)
df_test.columns = df_test.columns.str.replace(' ', '_')
df_test.columns = df_test.columns.str.replace('(','')
df_test.columns = df_test.columns.str.replace(')','')

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
modelo1 = smf.glm(formula = 'Churn ~ tenure + MonthlyCharges + TotalCharges + total_serv +\
                  gender_Male + SeniorCitizen_1 + Partner_Yes + Dependents_Yes +\
                  PhoneService_Yes + MultipleLines_No_phone_service +\
                  MultipleLines_Yes + InternetService_Fiber_optic +\
                  InternetService_No + OnlineSecurity_No_internet_service +\
                  OnlineSecurity_Yes + OnlineBackup_No_internet_service +\
                  OnlineBackup_Yes + DeviceProtection_No_internet_service +\
                  DeviceProtection_Yes + TechSupport_No_internet_service +\
                  TechSupport_Yes + StreamingTV_No_internet_service + StreamingTV_Yes +\
                  StreamingMovies_No_internet_service + StreamingMovies_Yes +\
                  Contract_One_year + Contract_Two_year + PaperlessBilling_Yes +\
                  PaymentMethod_Credit_card_automatic +\
                  PaymentMethod_Electronic_check + PaymentMethod_Mailed_check',
                  data = df_train,
                  family = sm.families.Binomial()).fit()
print(modelo1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 5634
Model:                            GLM   Df Residuals:                     5610
Model Family:                Binomial   Df Model:                           23
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2307.3
Date:                Mon, 16 Nov 2020   Deviance:                       4614.6
Time:                        04:51:04   Pearson chi2:                 5.58e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [52]:
modelo1 = LogisticRegression(penalty='l2', solver='newton-cg',max_iter=5000,multi_class='multinomial',n_jobs=-1)
y = df_train['Churn']
X = df_train.drop('Churn',axis=1)

In [21]:
modelo1.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='multinomial', n_jobs=-1, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
X_test = df_test

In [23]:
X.shape

(5634, 32)

In [559]:
X_test.shape

(1409, 32)

In [24]:
y_pred = modelo1.predict(X_test)

In [25]:
unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1097  312]]


In [31]:
df_submit = pd.DataFrame(zip(X_test['id'],y_pred),columns=['id','Churn'])
df_submit.to_csv('churn_02.csv',index = False, sep = ',')

,id,Churn
0,5027,0
1,1733,0
2,5384,1
3,6554,0
4,364,0


In [27]:
score = modelo1.score(X_test, y_test)
score

0.5067423704755145

In [568]:
score = modelo1.score(X, y)
score

0.8095491657791978

In [409]:
print(classification_report(y, modelo1.predict(X)))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      4139
           1       0.67      0.56      0.61      1495

    accuracy                           0.81      5634
   macro avg       0.76      0.73      0.74      5634
weighted avg       0.80      0.81      0.80      5634



In [412]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.77      0.62       734
           1       0.47      0.22      0.30       675

    accuracy                           0.51      1409
   macro avg       0.49      0.50      0.46      1409
weighted avg       0.49      0.51      0.46      1409



In [53]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=0)
modelCV = LogisticRegression(max_iter=500)
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X, y, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


10-fold cross validation average accuracy: 0.804


In [54]:
modelCV.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_pred_5 = modelCV.predict(X_test)

In [59]:
unique_elements, counts_elements = np.unique(y_pred_5, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1111  298]]


In [60]:
df_submit = pd.DataFrame(zip(X_test['id'],y_pred_5), columns=['id','Churn'])
df_submit.to_csv('churn_04.csv', index=False, sep=',')

In [33]:
# eliminando 'DeviceProtection_Yes','TotalCharges','gender_Male','PaymentMethod_Mailed_check',
# 'PaymentMethod_Credit_card_automatic', 'PhoneService_Yes',MultipleLines_No_phone_service',
# 'total_serv', 'OnlineBackup_Yes', 'Partner_Yes'
modelo2 = smf.glm(formula = 'Churn ~ tenure + MonthlyCharges +\
                  SeniorCitizen_1 + Dependents_Yes +\
                  MultipleLines_Yes + InternetService_Fiber_optic +\
                  InternetService_No + OnlineSecurity_No_internet_service +\
                  OnlineSecurity_Yes + OnlineBackup_No_internet_service +\
                  DeviceProtection_No_internet_service +\
                  TechSupport_No_internet_service +\
                  TechSupport_Yes + StreamingTV_No_internet_service + StreamingTV_Yes +\
                  StreamingMovies_No_internet_service + StreamingMovies_Yes +\
                  Contract_One_year + Contract_Two_year + PaperlessBilling_Yes +\
                  PaymentMethod_Electronic_check',
                  data = df_train,
                  family = sm.families.Binomial()).fit()
print(modelo2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 5634
Model:                            GLM   Df Residuals:                     5618
Model Family:                Binomial   Df Model:                           15
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2310.0
Date:                Mon, 16 Nov 2020   Deviance:                       4619.9
Time:                        04:40:10   Pearson chi2:                 5.61e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [96]:
print(np.exp(modelo1.params[1:]))

gender[T.Male]                               1.026847
Partner[T.Yes]                               0.892316
Dependents[T.Yes]                            0.863264
PhoneService[T.Yes]                          2.479269
MultipleLines[T.No phone service]            1.185884
MultipleLines[T.Yes]                         1.607139
InternetService[T.Fiber optic]              10.809657
InternetService[T.No]                        0.722873
OnlineSecurity[T.No internet service]        0.722873
OnlineSecurity[T.Yes]                        0.660572
OnlineBackup[T.No internet service]          0.722873
OnlineBackup[T.Yes]                          0.848400
DeviceProtection[T.No internet service]      0.722873
DeviceProtection[T.Yes]                      0.976475
TechSupport[T.No internet service]           0.722873
TechSupport[T.Yes]                           0.752458
StreamingTV[T.No internet service]           0.722873
StreamingTV[T.Yes]                           1.793100
StreamingMovies[T.No interne

### https://www.kaggle.com/funxexcel/p2-logistic-regression-hyperparameter-tuning

In [104]:
logModel = LogisticRegression()

In [105]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

In [106]:
# import GridSearch
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
best_clf = clf.fit(X,y)

In [ ]:
best_clf.best_estimator_

In [ ]:
print (f'Accuracy - : {best_clf.score(X,y):.3f}')

In [ ]:
### fim

### Fim

In [34]:
# eliminando as colunas 
modelo2 = LogisticRegression(penalty='l2', solver='newton-cg',max_iter=5000,multi_class='multinomial',n_jobs=-1)
y = df_train['Churn']
X = df_train.drop(['DeviceProtection_Yes','TotalCharges','gender_Male','PaymentMethod_Mailed_check',
                   'PaymentMethod_Credit_card_automatic', 'PhoneService_Yes','MultipleLines_No_phone_service',
                   'total_serv', 'OnlineBackup_Yes', 'Partner_Yes','Churn'],axis=1)

In [35]:
X.shape

(5634, 22)

In [374]:
y.shape

(5634,)

In [36]:
modelo2.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='multinomial', n_jobs=-1, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [208]:
y_pred = modelo2.predict_proba(X)

In [37]:
X_test = df_test.drop(['DeviceProtection_Yes','TotalCharges','gender_Male','PaymentMethod_Mailed_check',
                       'PaymentMethod_Credit_card_automatic', 'PhoneService_Yes','MultipleLines_No_phone_service',
                       'total_serv', 'OnlineBackup_Yes', 'Partner_Yes'],
                      axis=1)

In [38]:
X_test.shape

(1409, 22)

In [39]:
y_pred_2 = modelo2.predict(X_test)

In [40]:
unique_elements, counts_elements = np.unique(y_pred_2, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1096  313]]


In [282]:
y_test = df_sample_submission['Churn']

In [283]:
acuracia = accuracy_score(y_test, y_pred)
acuracia

0.5074520936834634

In [287]:
y.value_counts()

0    4139
1    1495
Name: Churn, dtype: int64

In [288]:
y_test.value_counts()

0    734
1    675
Name: Churn, dtype: int64

In [586]:
print(classification_report(y, modelo2.predict(X)))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      4139
           1       0.67      0.56      0.61      1495

    accuracy                           0.81      5634
   macro avg       0.76      0.73      0.74      5634
weighted avg       0.80      0.81      0.80      5634



In [587]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.78      0.62       734
           1       0.47      0.22      0.30       675

    accuracy                           0.51      1409
   macro avg       0.50      0.50      0.46      1409
weighted avg       0.50      0.51      0.47      1409



In [ ]:
df_submit = pd.DataFrame(y_pred, columns = ['Churn'])

In [ ]:
df_submit.value_counts()

Churn
0        1119
1         290
dtype: int64

In [ ]:
y_test.value_counts()

0    734
1    675
Name: Churn, dtype: int64

In [ ]:
plot_roc_curve(y, y_pred)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression(max_iter=500)
logreg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
y_pred_3 = logreg.predict(X_test)

In [48]:
unique_elements, counts_elements = np.unique(y_pred_3, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1105  304]]


In [392]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_pred_2)))

Accuracy of logistic regression classifier on test set: 1.00


In [41]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=0)
modelCV = LogisticRegression(max_iter=500)
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X, y, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


10-fold cross validation average accuracy: 0.801


In [42]:
modelCV.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_pred_4 = modelCV.predict(X_test)

In [45]:
unique_elements, counts_elements = np.unique(y_pred_4, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1105  304]]


In [49]:
df_submit = pd.DataFrame(zip(X_test['id'],y_pred_4),columns=['id','Churn'])
df_submit.to_csv('churn_03.csv',index = False, sep = ',')

In [417]:
print(classification_report(y_test, y_pred_3))

              precision    recall  f1-score   support

           0       0.52      0.78      0.62       734
           1       0.47      0.21      0.29       675

    accuracy                           0.51      1409
   macro avg       0.49      0.49      0.45      1409
weighted avg       0.49      0.51      0.46      1409

